# File Contents Converter

### バイナリファイル内の文字列を任意に置換するプログラム

#### 01. read確認

In [2]:
fr = open('test_01.DMP', 'rb')
read_data = fr.read()
print(read_data)
fr.close()

b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?'


#### 02. write確認

In [3]:
fw = open('write_01.DMP', 'wb')
fw.write(read_data)
fw.close()

#### 03. 置換の確認

In [5]:
for i in (b'\x00', b'\x10', b'\x20', b'\x30'):
    read_data = read_data.replace(i, b'\xAA')
fc = open('convert_01.DMP', 'wb')
fc.write(read_data)
fc.close()